In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, save_model

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# https://scortex.io/batch-norm-folding-an-easy-way-to-improve-your-network-speed/
def get_folded_weights(conv_weights, batch_norm_weights, epsilon=1e-3):
    #original
    # gamma = batch_norm_weights[0].reshape((1,1,1,batch_norm_weights[0].shape[0]))
    #my method
    gamma = batch_norm_weights[0].reshape((batch_norm_weights[0].shape[0]))
    beta = batch_norm_weights[1]
    mean = batch_norm_weights[2]
    # original
    # variance = batch_norm_weights[3].reshape((1,1,1,batch_norm_weights[3].shape[0]))
    #my method
    variance = batch_norm_weights[3].reshape((batch_norm_weights[3].shape[0]))
    new_weight = conv_weights[0] * gamma / np.sqrt(variance+epsilon)
    
    new_bias = beta + (conv_weights[1]- mean) * gamma / np.sqrt(variance + epsilon)

    #my addendum to fix shape from (1, 1, 1, 32) to (32,)
    return new_weight, new_bias

# Use this for Linear Sequential Models ONLY
1. Loops through all layers in original model and duplicate all layers except BatchNorm and Dropout
2. Loops through layers in new model and grabs sequential Convolutions and BatchNorm weights and biases and then computes linear transformation
3. Replaces applicable convolution layer weights with newly computed folded weights and biases

* Since dense layers can be represented as 1x1 convolutions, dense layers can be folded with batchnorm

In [ ]:
def FoldBN(model, new_model):
    '''Folds Batchnormalization layers into nearest convolution
    Limitation is that activation cannot sit between conv and BN layer
    '''
    for idx, layer in enumerate(model.layers):
        #Layer must be conv2D and have none/linear activation
        if layer.__class__.__name__ == 'Conv2D':
            if layer.activation.__name__=='linear':
                if model.layers[idx+1].__class__.__name__ == 'BatchNormalization':
                    Conv = layer
                    ConvWeights = Conv.get_weights()

                    BatchNorm = model.layers[idx+1]
                    NormWeights = BatchNorm.get_weights()

                    ConvWeight, ConvBias = get_folded_weights(ConvWeights, NormWeights)

                    for new_layer in new_model.layers:
                        if layer.name == new_layer.name:
                            print(f'Updated {layer.name} weights and biases')
                            new_layer.set_weights([ConvWeight, ConvBias])
                            break
        
            elif model.layers[idx-1].__class__.__name__ == 'BatchNormalization':
                Conv = layer
                ConvWeights = Conv.get_weights()

                BatchNorm = model.layers[idx-1]
                NormWeights = BatchNorm.get_weights()

                ConvWeight, ConvBias = get_folded_weights(ConvWeights, NormWeights)

                for new_layer in new_model.layers:
                    if layer.name == new_layer.name:
                        print(f'Updated {layer.name} weights and biases')
                        new_layer.set_weights([ConvWeight, ConvBias])
                        break
    return new_model

def DuplicateModelLinear(model, without=['BatchNormalization', 'Dropout']):
    '''Duplicates model other than layers listed in without argument
    '''
    new_model = tf.keras.Sequential([])
    for layer in model.layers:
        if not layer.__class__.__name__ in without:
            new_model.add(layer)
    optimizer = model.optimizer
    loss = model.loss

    new_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    print('x')
    return new_model

In [ ]:
MODEL = load_model('')
new_model = DuplicateModelLinear(MODEL)
new_model = FoldBN(MODEL, new_model)

# Generalized BatchNorm Folding Technique
Can be used on Linear and NonLinear Models
1. Redefine model without BatchNorm or Dropout Layers and compile
* Make sure model names match original models
2. Name layers in pairs of (Conv2D, BatchNorm) in a list
3. Copy original weights of kept layers from original to new model
4. Loop through name list and fold pair together and assign folded weights and biases into the new model

In [ ]:
def copy_original_weights(model, new_model):
    '''Takes new model and copies all relevant original weights into it
    '''
    for new_layer in new_model.layers:
        old_layer = model.get_layer(new_layer.name)
        new_layer.set_weights(old_layer.get_weights())
    return new_model

def FoldBN(model, new_model, layers):
    '''Fold BatchNorm layers into decided convolution

    Args:
        model: original keras model
        new_model: new keras model with batch norm layers removed
        layers: tuple of tuples (Conv, BatchNorm)of layers that want to be
            matched by name
    '''
    for layer_names in layers:
        Conv = model.get_layer(layer_names[0])
        Norm = model.get_layer(layer_names[1])    
        Norm_weights = Norm.get_weights()
        Conv_weights = Conv.get_weights()

        NewWeight, NewBias = get_folded_weights(Conv_weights, Norm_weights)
        layer = new_model.get_layer(Conv.name)
        layer.set_weights([NewWeight, NewBias])
        print(f'Layer weights from {layer_names[0]} & {layer_names[1]} are folded')
    return new_model

In [ ]:
####################################
#Import Relevant Layers
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Activation, Conv2D, Dense, Flatten, InputLayer, concatenate
model = load_model('functional_split_net')
# INPUT_SHAPE = (28,28,1)
# NEW_MODEL = tf.keras.Sequential(
#     [
#     InputLayer(input_shape=INPUT_SHAPE),
#     Conv2D(32, kernel_size=(3, 3), name='C1', activation=None),
#     Activation('relu'),
#     Conv2D(32, kernel_size=(3, 3), name='C2', activation=None),
#     Activation('relu'),
#     Conv2D(32, kernel_size=(3, 3), name='C3', activation=None),
#     Activation('relu'),
#     Conv2D(32, kernel_size=(3, 3), name='C4', activation=None),
#     Activation('relu'),
#     Conv2D(32, kernel_size=(3, 3), name='C5', activation=None),
#     Activation('relu'),
#     Conv2D(32, kernel_size=(3, 3), name='C6', activation=None),
#     Activation('relu'),
#     Conv2D(32, kernel_size=(3, 3), name='C7', activation=None),
#     Activation('relu'),
#     Flatten(name='flatten'),
#     Dense(10, activation="softmax", name='classifier'),
#     ]
# )
# LAYERS = (('C1','BN1'),('C2','BN2'),('C3','BN3'),('C4','BN4'),('C5','BN5'),('C6','BN6'),('C7','BN7'))
input = Input(shape=(28,28,1))
x = Conv2D(32, kernel_size=(3, 3), name='C1', activation=None)(input)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=(3, 3), name='C2', activation=None)(x)
x = Activation('relu')(x)

y = Conv2D(64, kernel_size=(3, 3), name='Cy1', activation=None)(x)
y = Activation('relu')(y)
y = Conv2D(64, kernel_size=(3, 3), name='Cy2', activation=None)(y)
y = Activation('relu')(y)

z = Conv2D(32, kernel_size=(3, 3), name='Cz1', activation=None)(x)
z = Activation('relu')(z)
z = Conv2D(32, kernel_size=(3, 3), name='Cz2', activation=None)(z)
z = Activation('relu')(z)

x = concatenate([y, z])
x = Conv2D(32, kernel_size=(3, 3), name='C3', activation=None)(x)
x = Activation('relu')(x)
x = Flatten(name='flatten')(x)
out = Dense(10, activation='softmax', name='classifier')(x)

NEW_MODEL = Model(inputs=[input], outputs=[out])
LAYERS = (('C1','BN1'),('C2','BN2'),('C3','BN3'),('Cy1','BNy1'),('Cy2','BNy2'),('Cz1','BNz1'),('Cz2','BNz2'))
######################################
optimizer = model.optimizer
loss = model.loss
NEW_MODEL.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

new_model_with_weights  = copy_original_weights(model, NEW_MODEL)
NEW_MODEL = FoldBN(model, NEW_MODEL, LAYERS)

In [ ]:
save_model(NEW_MODEL, 'functional_split_folded_wBN')